In [1]:
import numpy as np
import pandas as pd
from Score import rouge_1,rouge_2,rouge_l,bleu_4,score

from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizer, BertModel, BertConfig

In [34]:
# 加载本地Bert模型
# Load the BERT model and tokenizer
config = BertConfig.from_json_file("../Bert_model/Bert/config.json")
tokenizer = BertTokenizer.from_pretrained('../Bert_model/Bert/')
bert = BertModel.from_pretrained("../Bert_model/Bert/", config=config)

In [35]:
# 获取词向量
def get_word_embeddings(text, model, token):
    text = "CLS" + text + "SEP"
    encoded_inputs = token(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # encoded_inputs = {k: v.to(device) for k, v in encoded_inputs.items()}

    # Pretrain the text
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Get the CLS token embeddings
    # 将cls标记的向量作为整个句子向量
    word_embedding = outputs[0][:, 0, :]

    return word_embedding

In [36]:
def semantic_similarity(answer_1, answer_2, model=bert, token=tokenizer):
    a = 0
    try:
        s1 = get_word_embeddings(answer_1, model, token)
        s2 = get_word_embeddings(answer_2, model, token)
        # 计算余弦相似度
        similarity = cosine_similarity(s1.cpu(), s2.cpu())
        a = round(np.mean(np.mean(similarity, axis=1)), 4)
        # print(f"第{i+1}段文本的语义相似度为: {a:.4f}")
    except Exception as e3:
        print(f"相似度计算的错误为{e3}")

    return a

读取数据

In [81]:
data = pd.read_csv('data_1000.csv').iloc[:,:4]

In [77]:
#所有列名
data.columns.tolist()

['问题', '医生回答', 'ChatGPT回答', '医生']

获取ChatGPT回答

In [69]:
from openai import OpenAI
# from time import sleep
client = OpenAI(
    # base_url="https://api.xiaoai.plus/v1",
    base_url="https://api.xty.app/v1",
    # api_key='sk-xPbZWfdDebqGfo8q2aDbF8D0Dc0a4b5fA0A797Ec626c2c50',
    api_key='sk-nAjP7DTOL82aDNKe9a35AfB0F21e4c0887F00b96E9B4A043',
)

In [71]:
def generate_response(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user","content": user_input,}
        ],
        model="gpt-3.5-turbo",
        max_tokens=512,
        temperature=0.7,
        # n=2,
        top_p=0.5
        )
    message=chat_completion.choices[0].message.content
    
    # message=chat_completion.choices[0]
    return message.strip().replace("\n", "")

In [78]:
#创建新数据集存储生成的多个答案
data_GPT=pd.DataFrame(columns=['问题','医生回答','GPTanswer_1','c1','GPTanswer_2','c2','GPTanswer_3','c3','GPTanswer_4','c4','GPTanswer_5','c5'])

In [79]:
# GPTresults=[]
# c=[]
answer_name=['GPTanswer_1','GPTanswer_2','GPTanswer_3','GPTanswer_4','GPTanswer_5']
c_name=['c1','c2','c3','c4','c5']

In [87]:
for index in range(961,len(data)):
#保存回答和问题在新的数据集
    data_GPT.at[index,'问题']=data['问题'][index]
    data_GPT.at[index,'医生回答']=data['医生回答'][index]
    
    question= data["问题"][index]
    # instruction="针对给出的问题，给出更可靠的答案"
    
    answer1=generate_response(question)
    # c1=round(score([question],[answer1]).iloc[0,0],4)
    
    data_GPT.at[index,answer_name[0]]=answer1
    # data_GPT.at[index,c_name[0]]=c1
    
    # in_put=f"问题：{question}\n回答1：{answer1}\n得分：{c1}\n"
    
    for i in range(1,5):
        try:
            answer=generate_response(question)
            # c=round(score([question],[answer]).iloc[0,0],4)
            
            data_GPT.at[index,answer_name[i]]=answer
            # data_GPT.at[index,c_name[i]]=c
            
            
            # in_put=in_put+f"回答{i+1}:{answer}\n得分：{c}\n"
                
            print(f"第{index}次问答")
        except Exception as e:
            print(f"Error occurred for question at index {index}: {e}")
            answer = None
            # c=None
data_GPT.to_csv('chatgpt问答_5_answer_2.csv')

第961次问答
第961次问答
第961次问答
第961次问答
第962次问答
第962次问答
第962次问答
第962次问答
第963次问答
第963次问答
第963次问答
第963次问答
第964次问答
第964次问答
第964次问答
第964次问答
第965次问答
第965次问答
第965次问答
第965次问答
第966次问答
第966次问答
第966次问答
第966次问答
第967次问答
第967次问答
第967次问答
第967次问答
第968次问答
第968次问答
第968次问答
第968次问答
第969次问答
第969次问答
第969次问答
第969次问答
第970次问答
第970次问答
第970次问答
第970次问答
第971次问答
第971次问答
第971次问答
第971次问答
第972次问答
第972次问答
第972次问答
第972次问答
第973次问答
第973次问答
第973次问答
第973次问答
第974次问答
第974次问答
第974次问答
第974次问答
第975次问答
第975次问答
第975次问答
第975次问答
第976次问答
第976次问答
第976次问答
第976次问答
第977次问答
第977次问答
第977次问答
第977次问答
第978次问答
第978次问答
第978次问答
第978次问答
第979次问答
第979次问答
第979次问答
第979次问答
第980次问答
第980次问答
第980次问答
第980次问答
第981次问答
第981次问答
第981次问答
第981次问答
第982次问答
第982次问答
第982次问答
第982次问答
第983次问答
第983次问答
第983次问答
第983次问答
第984次问答
第984次问答
第984次问答
第984次问答
第985次问答
第985次问答
第985次问答
第985次问答
第986次问答
第986次问答
第986次问答
第986次问答
第987次问答
第987次问答
第987次问答
第987次问答
第988次问答
第988次问答
第988次问答
第988次问答
第989次问答
第989次问答
第989次问答
第989次问答
第990次问答
第990次问答
第990次问答
第990次问答
第991次问答
第991次问答
第991次问答
第991次问答
第992次问答


In [89]:
data_GPT

,问题,医生回答,GPTanswer_1,c1,GPTanswer_2,c2,GPTanswer_3,c3,GPTanswer_4,c4,GPTanswer_5,c5
0,被动抬腿试验的意义,直腿抬高试验是脊柱检查时特殊的试验，主要用于诊断腰椎间盘突出症，也可以诊断坐骨神经痛。而一般...,被动抬腿试验是一种常见的神经系统检查方法，用于评估腰椎神经根的功能和可能的神经根受压情况。这...,NaN,被动抬腿试验是一种常见的神经系统检查方法，用于评估下肢神经和脊髓的功能。这项测试的主要目的是...,NaN,被动抬腿试验是一种常见的神经学检查方法，用于评估下肢神经系统的功能。这项测试可以帮助医生诊断...,NaN,被动抬腿试验是一种常见的神经系统检查方法，通常用于评估神经系统的功能和异常。这项测试主要用于...,NaN,被动抬腿试验是一种常见的神经系统检查方法，用于评估患者下肢神经系统的功能和可能存在的问题。这...,NaN
1,哺乳期阴道炎用什么药好得快,哺乳期是特殊的时期，因为涉及到母乳喂养，所以说用药方面一定是要谨慎的。哺乳期得了阴道炎的建议...,如果你正在哺乳期并怀疑患有阴道炎，最好的做法是先咨询医生。哺乳期间使用药物需要特别小心，因为...,NaN,哺乳期阴道炎需要谨慎治疗，因为一些药物可能会对宝宝产生影响。建议你立即咨询医生，他们可以根据...,NaN,哺乳期阴道炎需要及时就医，最好遵医嘱使用药物治疗。通常医生会根据病情的严重程度和病因来选择合...,NaN,哺乳期阴道炎需要咨询医生以获取正确的诊断和治疗建议。一般情况下，医生可能会建议使用抗真菌药物...,NaN,哺乳期阴道炎需要及时就医，最好遵医嘱使用药物治疗。通常医生会根据病情的严重程度和病因来选择合...,NaN
2,打完水光针后几天可以敷医用面膜,从原则上来说，打完水光针后十天左右就可以敷医用面膜了，不过不同患者情况不同。正常情况下，患者...,打完水光针后一般建议等待至少24小时再进行其他面部护理，包括敷面膜。这是因为水光针治疗后，皮...,NaN,打完水光针后，通常建议在接下来的两到三天内避免使用任何可能会刺激皮肤的产品，包括面膜。这是因...,NaN,打完水光针后，通常建议等待至少24小时后再进行面部护理，以避免刺激皮肤。一般来说，可以在水光...,NaN,通常情况下，接受水光针治疗后，皮肤可能会出现轻微的红肿或刺痛感，因此需要给皮肤一些时间来恢复...,NaN,打完水光针后，通常建议等待至少24小时后再进行面部护理，以避免刺激皮肤。一般来说，可以在水光...,NaN
3,月子里腿受风了怎么补救,月子里腿受风的补救方法可以参考以下几种：1.如果有腿部疼痛、恶风怕冷、或有腰酸软冷痛等症状，...,月子期间腿受风了确实需要特别小心和谨慎。首先，要确保保持温暖，可以穿着长裤和袜子，并避免让受...,NaN,月子期间要特别小心保暖，因为身体虚弱，容易受风。如果腿受风了，以下是一些建议来帮助缓解不适：...,NaN,月子期间特别需要注意保暖，尤其是腿部。如果腿部受了风，可以考虑以下方法来缓解：1. **保持...,NaN,月子里腿受风了可能会引起不适，但具体的补救方法可能会因个人情况而异。这里有几种可能的方法：1...,NaN,月子期间腿受风了可能会导致身体不适，建议您采取以下措施来补救：1. 保持腿部温暖：穿上保暖的...,NaN
4,盆腔炎是什么病菌造成的症状表现,盆腔炎是指女性内生殖器及其周围的炎症性疾病，常常由细菌、病毒、真菌等病原体引起。常见的病菌包...,盆腔炎是由细菌感染引起的盆腔内器官炎症性疾病。常见的症状包括下腹部疼痛、盆腔不适、发热、异常...,NaN,盆腔炎是由细菌感染引起的盆腔内器官炎症性疾病。常见的症状包括下腹部疼痛、盆腔不适、发热、异常...,NaN,盆腔炎通常是由细菌感染引起的，最常见的是性传播疾病（如淋病和非淋菌性尿道炎）引起的细菌感染。...,NaN,盆腔炎是一种感染性疾病，通常由细菌引起，最常见的细菌包括阴道中的常见细菌以及性传播疾病（ST...,NaN,盆腔炎是一种由细菌感染引起的炎症性疾病，通常涉及子宫、输卵管和附件等盆腔器官。常见的症状包括...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,激光脱毛可以干净吗,激光脱毛一般可以干净，可能需要多次激光脱毛操作。激光脱毛可以破坏毛发根部的毛囊，使其失去活性...,激光脱毛是一种有效的方法，可以帮助去除多余的毛发。激光脱毛可以在较长的时间内减少毛发的生长，...,NaN,是的，激光脱毛通常可以达到很干净的效果。激光脱毛利用激光能量破坏毛囊，阻止毛发再生，因此可以...,NaN,激光脱毛通常可以达到相当干净的效果。激光脱毛利用激光能量破坏毛囊，从而减少甚至永久性地减少毛...,NaN,激光脱毛是一种有效的方法，可以帮助去除多余的毛发。激光脱毛可以在较长的时间内减少毛发的生长，...,NaN,激光脱毛通常可以提供相当干净的结果。激光脱毛使用激光光束瞄准毛囊，将光能转化为热能，从而摧毁...,NaN
996,哺乳期怀孕几天有反应,在哺乳期间，如果怀孕了，有很多人因为这段时间太大意，以为没有来月经就不会发生卵泡排出引起怀孕...,哺乳期怀孕后，一般来说，身体可能会出现一些早期怀孕的征兆，但这些征兆在不同的人身上表现可能会...,NaN,一般来说，怀孕后大约6到8周左右，哺乳期妈妈可能会开始出现怀孕的一些早期症状，比如乳房胀痛、...,NaN,一般来说，怀孕后大约6到8周左右，哺乳期妈妈可能会开始出现怀孕的一些早期症状，比如乳房胀痛、...,NaN,一般来说，怀孕后大约6到8周左右，哺乳期妈妈可能会开始出现怀孕的一些早期症状，比如乳房胀痛、...,NaN,一般来说，怀孕后的早期反应在怀孕后两周到一个月之间开始出现，这通常是因为怀孕后体内激素水平的...,NaN
997,小孩眼球有红血丝,小孩眼球有红血丝可能是结膜炎、过敏、咳嗽等因素导致的。1、结膜炎：当小孩的结膜感染时，可能会...,小孩眼睛出现红血丝可能有多种原因。这可能是由于过度疲劳、缺乏睡眠、过敏、眼睛受伤、干燥的环境...,NaN,小孩眼球出现红血丝可能有几种原因。常见的包括：1. **眼睛受到刺激**：例如长时间盯着屏幕...,NaN,小孩眼球有红血丝可能有多种原因，包括但不限于：1. **眼睛疲劳**：过度用眼、长时间盯着屏...,NaN,小孩眼球有红血丝可能有多种原因，其中一些可能是正常的，而另一些可能需要医生的关注。红血丝可能...,NaN,小孩眼球有红血丝可能是由于多种原因引起的，包括但不限于：1. **眼睛疲劳：** 长时间盯着...,NaN
998,老人心脏早搏该怎么办,老人心脏早搏的治疗，要看早搏的严重程度，是偶发还是频发早搏。心脏早搏主要包括房性早搏和室性早...,老人心脏早搏是一种心律失常，如果老人出现心脏早搏，应该及时就医。以下是一些建议：1. 就医：...,NaN,老人心脏早搏可能是心脏节律异常的一种表现，但不必过于担心，因为这种情况在老年人中相对常见。然...,NaN,老人心脏早搏是一种心律失常，如果老人出现心脏早搏，应该及时就医。以下是一些建议：1. 就医：...,NaN,老人心脏早搏是一种心律失常，如果老人出现心脏早搏，应该及时就医。以下是一些建议：1. 就医：...,NaN,老年人的心脏早搏可能是由多种因素引起的，包括心血管疾病、电解质失衡、药物副作用等。如果老年人...,NaN


选择出最佳的回答

In [138]:
c1=score(data_GPT['问题'],data_GPT['GPTanswer_1'])
c2=score(data_GPT['问题'],data_GPT['GPTanswer_2'])
c3=score(data_GPT['问题'],data_GPT['GPTanswer_3'])
c4=score(data_GPT['问题'],data_GPT['GPTanswer_4'])
c5=score(data_GPT['问题'],data_GPT['GPTanswer_5'])

C:\Users\mi\PycharmProjects\textAnalysis\optimization_method\Score.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  answer_embedding = torch.unsqueeze(torch.tensor(answer_embedding), dim=1).numpy()


32/32 [==============================] - 5s 101ms/step


In [139]:
data_GPT['c1']=c1
data_GPT['c2']=c2
data_GPT['c3']=c3
data_GPT['c4']=c4
data_GPT['c5']=c5

In [185]:
data_GPT.to_csv('chatgpt问答_5_answer_3.csv')

每轮最佳答案

In [146]:
# 创建一个空dataframe来存储每个问题的最佳答案
best_answers = pd.DataFrame(columns=['L1','s1','L2','s2','L3','s3', 'L4','s4','L5','s5'])

In [147]:
#第一轮
best_answers['L1']=data_GPT['GPTanswer_1']

In [148]:
s1=[]
for i in range(len(best_answers['L1'])):
    s1.append(semantic_similarity(best_answers['L1'][i],data_GPT['医生回答'][i]))
best_answers['s1']=s1

In [151]:
len(best_answers[best_answers['s1']>=0.9])

250

2-5轮

In [179]:
#第4轮
c=[]
for i in range(1000):
    c.append(tuple([c1.iloc[i,0],c2.iloc[i,0],c3.iloc[i,0],c4.iloc[i,0],c5.iloc[i,0]]))

In [180]:
#找出C中每个元组中数据最大值的下标
max_index=[]

for i in range(len(c)):
    max_index.append(c[i].index(max(c[i])))
best_answers['L5_index']=max_index

In [181]:
#按照max_index选出最佳的答案
best_answer=[]
for i in range(len(max_index)):
    if max_index[i]==0:
        best_answer.append(data_GPT['GPTanswer_1'][i])
    elif max_index[i]==1:
        best_answer.append(data_GPT['GPTanswer_2'][i]) 
    elif max_index[i]==2:
        best_answer.append(data_GPT['GPTanswer_3'][i])
    elif max_index[i]==3:
        best_answer.append(data_GPT['GPTanswer_4'][i])
    else:
        best_answer.append(data_GPT['GPTanswer_5'][i])
best_answers['L5']=best_answer

In [182]:
#计算最佳答案与医生回答的相似度
s5=[]
for i in range(len(best_answers['L5'])):
    s5.append(semantic_similarity(best_answer[i],data_GPT['医生回答'][i]))
best_answers['s5']=s5

In [183]:
len(best_answers[best_answers['s5']>=0.9])

340

In [184]:
best_answers.to_csv('best_answers.csv')

指标分析

In [3]:
data_GPT=pd.read_csv('chatgpt问答_5_answer_3.csv')

In [19]:
data_GPT.head()

,Unnamed: 0,问题,医生回答,GPTanswer_1,c1,GPTanswer_2,c2,GPTanswer_3,c3,GPTanswer_4,c4,GPTanswer_5,c5
0,0,被动抬腿试验的意义,直腿抬高试验是脊柱检查时特殊的试验，主要用于诊断腰椎间盘突出症，也可以诊断坐骨神经痛。而一般...,被动抬腿试验是一种常见的神经系统检查方法，用于评估腰椎神经根的功能和可能的神经根受压情况。这...,0.392354,被动抬腿试验是一种常见的神经系统检查方法，用于评估下肢神经和脊髓的功能。这项测试的主要目的是...,0.546266,被动抬腿试验是一种常见的神经学检查方法，用于评估下肢神经系统的功能。这项测试可以帮助医生诊断...,0.573243,被动抬腿试验是一种常见的神经系统检查方法，通常用于评估神经系统的功能和异常。这项测试主要用于...,0.469839,被动抬腿试验是一种常见的神经系统检查方法，用于评估患者下肢神经系统的功能和可能存在的问题。这...,0.503650
1,1,哺乳期阴道炎用什么药好得快,哺乳期是特殊的时期，因为涉及到母乳喂养，所以说用药方面一定是要谨慎的。哺乳期得了阴道炎的建议...,如果你正在哺乳期并怀疑患有阴道炎，最好的做法是先咨询医生。哺乳期间使用药物需要特别小心，因为...,0.907264,哺乳期阴道炎需要谨慎治疗，因为一些药物可能会对宝宝产生影响。建议你立即咨询医生，他们可以根据...,0.895529,哺乳期阴道炎需要及时就医，最好遵医嘱使用药物治疗。通常医生会根据病情的严重程度和病因来选择合...,0.642914,哺乳期阴道炎需要咨询医生以获取正确的诊断和治疗建议。一般情况下，医生可能会建议使用抗真菌药物...,0.540635,哺乳期阴道炎需要及时就医，最好遵医嘱使用药物治疗。通常医生会根据病情的严重程度和病因来选择合...,0.760852
2,2,打完水光针后几天可以敷医用面膜,从原则上来说，打完水光针后十天左右就可以敷医用面膜了，不过不同患者情况不同。正常情况下，患者...,打完水光针后一般建议等待至少24小时再进行其他面部护理，包括敷面膜。这是因为水光针治疗后，皮...,0.650094,打完水光针后，通常建议在接下来的两到三天内避免使用任何可能会刺激皮肤的产品，包括面膜。这是因...,0.648843,打完水光针后，通常建议等待至少24小时后再进行面部护理，以避免刺激皮肤。一般来说，可以在水光...,0.537940,通常情况下，接受水光针治疗后，皮肤可能会出现轻微的红肿或刺痛感，因此需要给皮肤一些时间来恢复...,0.537047,打完水光针后，通常建议等待至少24小时后再进行面部护理，以避免刺激皮肤。一般来说，可以在水光...,0.540644
3,3,月子里腿受风了怎么补救,月子里腿受风的补救方法可以参考以下几种：1.如果有腿部疼痛、恶风怕冷、或有腰酸软冷痛等症状，...,月子期间腿受风了确实需要特别小心和谨慎。首先，要确保保持温暖，可以穿着长裤和袜子，并避免让受...,0.463220,月子期间要特别小心保暖，因为身体虚弱，容易受风。如果腿受风了，以下是一些建议来帮助缓解不适：...,0.604350,月子期间特别需要注意保暖，尤其是腿部。如果腿部受了风，可以考虑以下方法来缓解：1. **保持...,0.515050,月子里腿受风了可能会引起不适，但具体的补救方法可能会因个人情况而异。这里有几种可能的方法：1...,0.566498,月子期间腿受风了可能会导致身体不适，建议您采取以下措施来补救：1. 保持腿部温暖：穿上保暖的...,0.919461
4,4,盆腔炎是什么病菌造成的症状表现,盆腔炎是指女性内生殖器及其周围的炎症性疾病，常常由细菌、病毒、真菌等病原体引起。常见的病菌包...,盆腔炎是由细菌感染引起的盆腔内器官炎症性疾病。常见的症状包括下腹部疼痛、盆腔不适、发热、异常...,0.907344,盆腔炎是由细菌感染引起的盆腔内器官炎症性疾病。常见的症状包括下腹部疼痛、盆腔不适、发热、异常...,0.896153,盆腔炎通常是由细菌感染引起的，最常见的是性传播疾病（如淋病和非淋菌性尿道炎）引起的细菌感染。...,0.311302,盆腔炎是一种感染性疾病，通常由细菌引起，最常见的细菌包括阴道中的常见细菌以及性传播疾病（ST...,0.586647,盆腔炎是一种由细菌感染引起的炎症性疾病，通常涉及子宫、输卵管和附件等盆腔器官。常见的症状包括...,0.932705


In [2]:
best_answers = pd.read_csv('best_answers.csv')

In [6]:
#第1轮
rouge_1_score=rouge_1(best_answers['L5'],data_GPT['医生回答'])
rouge_2_score=rouge_2(best_answers['L5'],data_GPT['医生回答'])
rouge_l_score=rouge_l(best_answers['L5'],data_GPT['医生回答'])
bleu_4_score=bleu_4(best_answers['L5'],data_GPT['医生回答'])

print('rouge-1:',rouge_1_score)
print('rouge-2:',rouge_2_score)
print('rouge-l:',rouge_l_score)
print('bleu-4:',bleu_4_score)

rouge-1: 0.34484590338388293
rouge-2: 0.12606903279267062
rouge-l: 0.24907908994740643
bleu-4: 0.02754092067329885


In [7]:
np.mean(best_answers['s5'])

0.8727442999999999

#最后一轮的features

In [8]:
data=pd.read_csv('data_1000.csv')

In [50]:
all_data=pd.read_csv(r'C:\Users\mi\PycharmProjects\textAnalysis\特征指标量化\chat_data_no_zero_label.csv')

In [51]:
#以问题列为标准，选出all_data中与data中问题相同的行

old_data=all_data[all_data['问题'].isin(data['问题'])]

In [55]:
old_data.describe()

,len_text,num_sentences,num_words,num_stopwords,num_punctuations,num_short_sentences,num_adjectives,num_verbs,num_nouns,num_adverbs,num_keywords,text_entropy,num_medical_terms,medical_keywords_ratio,keywords_overlap,semantic_similarity,num_positive_words,num_native_words,sentiment_polarity,label
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,194.650000,6.244000,108.504000,39.325000,16.604000,15.549000,4.660000,28.605000,28.889000,3.613000,69.179000,5.524724,3.990000,0.060519,0.087964,0.735733,29.626000,15.918000,0.810865,0.255000
std,90.562504,2.334934,48.654242,17.887911,8.781361,8.720645,3.061335,14.475514,13.994232,2.287458,33.143981,0.584827,2.994473,0.038674,0.050045,0.046931,15.880501,8.515033,0.346856,0.436079
min,40.000000,2.000000,25.000000,9.000000,3.000000,2.000000,1.000000,5.000000,1.000000,1.000000,9.000000,3.169900,1.000000,0.006000,0.012000,0.577600,4.000000,1.000000,0.000010,0.000000
25%,123.000000,5.000000,71.000000,27.000000,10.000000,9.000000,2.000000,18.000000,18.000000,2.000000,43.000000,5.129300,2.000000,0.032000,0.053000,0.706700,18.000000,10.000000,0.860420,0.000000
50%,166.000000,6.000000,95.000000,35.000000,14.000000,13.000000,4.000000,25.000000,26.000000,3.000000,60.000000,5.491900,3.000000,0.052000,0.077000,0.737350,26.000000,14.000000,0.999725,0.000000
75%,260.000000,8.000000,141.000000,48.250000,23.000000,21.250000,6.000000,38.000000,39.000000,5.000000,92.000000,5.977300,5.000000,0.080000,0.111000,0.769250,38.000000,21.000000,1.000000,1.000000
max,545.000000,18.000000,285.000000,133.000000,57.000000,56.000000,19.000000,90.000000,96.000000,16.000000,201.000000,6.977300,22.000000,0.280000,0.324000,0.871300,105.000000,59.000000,1.000000,1.000000


In [56]:
old_data.to_csv('old_data.csv',index=False)

In [39]:
new_data=pd.DataFrame(columns=['问题','best_answer'])

In [40]:
new_data['问题']=data['问题']

new_data['best_answer']=best_answers['L5']

In [42]:
new_data.to_csv('new_data.csv',index=False)

In [43]:
#问题和最佳答案的相似度
similarity=[]
for i in range(len(new_data['问题'])):
    similarity.append(semantic_similarity(new_data['best_answer'][i],new_data['问题'][i]))
    print(f'问题{i+1}的相似度为{similarity[i]}')

问题1的相似度为0.6948999762535095
问题2的相似度为0.7113999724388123
问题3的相似度为0.7585999965667725
问题4的相似度为0.7160999774932861
问题5的相似度为0.7716000080108643
问题6的相似度为0.6905999779701233
问题7的相似度为0.7745000123977661
问题8的相似度为0.753000020980835
问题9的相似度为0.7621999979019165
问题10的相似度为0.7523999810218811
问题11的相似度为0.6668999791145325
问题12的相似度为0.8198999762535095
问题13的相似度为0.7669000029563904
问题14的相似度为0.6974999904632568
问题15的相似度为0.7645999789237976
问题16的相似度为0.6736999750137329
问题17的相似度为0.6622999906539917
问题18的相似度为0.7411999702453613
问题19的相似度为0.8036999702453613
问题20的相似度为0.7297000288963318
问题21的相似度为0.7124000191688538
问题22的相似度为0.7440000176429749
问题23的相似度为0.6919999718666077
问题24的相似度为0.7272999882698059
问题25的相似度为0.6589999794960022
问题26的相似度为0.7735000252723694
问题27的相似度为0.7669000029563904
问题28的相似度为0.7330999970436096
问题29的相似度为0.765999972820282
问题30的相似度为0.7418000102043152
问题31的相似度为0.7894999980926514
问题32的相似度为0.7301999926567078
问题33的相似度为0.7337999939918518
问题34的相似度为0.7125999927520752
问题35的相似度为0.7160999774932861
问题36的相似度为0.685699999332428
问题37

In [45]:
new_data_features=pd.read_csv('new_data_feature.csv')

In [47]:
new_data_features['semantic_similarity']=similarity

In [48]:
new_data_features.to_csv('new_data_feature.csv',index=False)

In [49]:
new_data_features.describe()

,Unnamed: 0,len_text,num_sentences,num_words,num_stopwords,num_punctuations,num_short_sentences,num_adjectives,num_verbs,num_nouns,num_adverbs,num_keywords,text_entropy,num_medical_terms,medical_keywords_ratio,keywords_overlap,semantic_similarity,num_positive_words,num_native_words,sentiment_polarity
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,217.926000,6.617000,120.246000,43.893000,18.727000,17.560000,4.962000,31.457000,31.923000,3.775000,76.353000,5.628748,4.161000,0.055009,0.082395,0.730016,32.756000,17.031000,0.827574
std,288.819436,89.150246,2.101125,46.123865,16.781215,8.731937,8.667498,3.096473,13.672494,13.839603,2.459761,32.468702,0.538853,3.164663,0.035962,0.046077,0.046593,15.188784,8.344679,0.341027
min,0.000000,36.000000,2.000000,19.000000,8.000000,3.000000,2.000000,0.000000,4.000000,3.000000,0.000000,6.000000,2.585000,0.000000,0.000000,0.000000,0.590200,2.000000,2.000000,0.000000
25%,249.750000,142.000000,5.000000,81.000000,32.000000,11.000000,10.000000,3.000000,20.000000,21.000000,2.000000,49.000000,5.209500,2.000000,0.029000,0.050000,0.697600,21.000000,11.000000,0.938480
50%,499.500000,205.000000,6.000000,113.000000,42.000000,17.500000,16.000000,4.000000,29.000000,30.000000,3.000000,71.000000,5.643900,4.000000,0.049500,0.072500,0.731200,30.000000,15.000000,0.999980
75%,749.250000,282.000000,8.000000,154.000000,54.000000,25.000000,24.000000,7.000000,42.000000,42.000000,5.000000,100.000000,6.044400,6.000000,0.074000,0.104000,0.763850,42.000000,22.000000,1.000000
max,999.000000,540.000000,15.000000,327.000000,121.000000,53.000000,53.000000,18.000000,84.000000,93.000000,18.000000,227.000000,6.965800,24.000000,0.237000,0.333000,0.870800,86.000000,49.000000,1.000000
